# 🚢 Titanic Survival Data Analysis  

This project is part of my data science internship at **Prodigy InfoTech**. In this task, I performed **data cleaning and exploratory data analysis (EDA)** on the famous **Titanic dataset** to uncover survival trends based on features like gender, age, passenger class, and more.

---

### 🧰 Tools Used:
- Python  
- Pandas  
- Seaborn  
- Matplotlib  

---

### 📌 Goals:
- Inspect and understand the structure of the dataset  
- Handle missing values and perform feature engineering  
- Explore and visualize patterns in survival rates  
- Analyze the impact of variables like **sex**, **Pclass**, **AgeGroup**, **Title**, and **FamilySize** on survival  

---

### 🔍 Key Tasks:
- Extracted new features like `Title`, `TicketPrefix`, `FamilySize`, and `FareBand`  
- Grouped rare categories under ‘Other’ for clarity  
- Created visualizations (bar plots, histograms, heatmaps) to reveal trends  
- Compared survival rates across age groups, titles, family categories, etc.


## Step 1: Import Libraries & Upload Dataset

We import necessary libraries and upload the Titanic dataset directly from our computer into the notebook. This way, we can start exploring our local data easily.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files

uploaded = files.upload()

titanic_df = pd.read_csv(list(uploaded.keys())[0])

titanic_df.head()


## Step 2: Data Inspection & Summary Statistics

We explore the dataset's size, column data types, and identify missing values. Summary statistics help us understand the numerical columns like Age and Fare, revealing potential data cleaning needs.


In [ ]:
print("Dataset shape:", titanic_df.shape)

print("\nInfo:")
titanic_df.info()

print("\nSummary statistics:")
titanic_df.describe()

print("\nMissing values per column:")
print(titanic_df.isnull().sum())


## Step 3: Handle Missing Values

We fill missing Age values with the median age and missing Embarked values with the most common port. The Cabin column has too many missing values and is dropped to keep the dataset clean.


In [ ]:
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].median())
titanic_df['Embarked'] = titanic_df['Embarked'].fillna(titanic_df['Embarked'].mode()[0])

if 'Cabin' in titanic_df.columns:
    titanic_df = titanic_df.drop(columns=['Cabin'])


## Step 4: Feature Engineering

We extract the passenger's Title from the Name column, grouping rare titles into 'Other' to reduce complexity. We create `FamilySize` to represent total family aboard, `AgeGroup` to categorize passengers by age ranges, and `FareBand` to segment fare prices into quartiles. These engineered features help reveal patterns in survival analysis.


In [ ]:
titanic_df['Title'] = titanic_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major',
               'Rev', 'Sir', 'Jonkheer', 'Dona']
titanic_df['Title'] = titanic_df['Title'].replace(rare_titles, 'Other')

titanic_df['FamilySize'] = titanic_df['SibSp'] + titanic_df['Parch'] + 1

bins = [0, 12, 20, 40, 60, 80]
labels = ['Child', 'Teen', 'Adult', 'Senior', 'Elder']
titanic_df['AgeGroup'] = pd.cut(titanic_df['Age'], bins=bins, labels=labels)

titanic_df['FareBand'] = pd.qcut(titanic_df['Fare'], 4, labels=[1, 2, 3, 4])


## Step 5: Survival Rates by Features

We calculate average survival rates grouped by Title, FamilySize, AgeGroup, and FareBand to identify how these factors influenced survival chances.


In [ ]:
import numpy as np

print(titanic_df.groupby('Title')['Survived'].mean())
print()
print(titanic_df.groupby('FamilySize')['Survived'].mean())
print()
print(titanic_df.groupby('AgeGroup')['Survived'].mean())
print()
print(titanic_df.groupby('FareBand')['Survived'].mean())


In [ ]:
print(titanic_df.groupby('AgeGroup', observed=True)['Survived'].mean())
print()
print(titanic_df.groupby('FareBand', observed=True)['Survived'].mean())


## Step 6: Visualize Survival Rates

Bar plots show how survival chances vary by Title, Family Size, Age Group, and Fare Band, helping us visually identify important patterns.


In [ ]:
sns.barplot(x='Title', y='Survived', data=titanic_df)
plt.title('Survival Rate by Title')
plt.show()

sns.barplot(x='FamilySize', y='Survived', data=titanic_df)
plt.title('Survival Rate by Family Size')
plt.show()

sns.barplot(x='AgeGroup', y='Survived', data=titanic_df)
plt.title('Survival Rate by Age Group')
plt.show()

sns.barplot(x='FareBand', y='Survived', data=titanic_df)
plt.title('Survival Rate by Fare Band')
plt.show()


## Step 7: Prepare Data for Modeling

We select relevant features, encode categorical variables using one-hot encoding, and split the data into training and testing sets to build a predictive model.


In [ ]:
from sklearn.model_selection import train_test_split

features = titanic_df.drop(['Survived', 'PassengerId', 'Name', 'Ticket'], axis=1)
target = titanic_df['Survived']

features = pd.get_dummies(features)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


## Step 8: Build and Evaluate Logistic Regression Model

We train a logistic regression model on the training data, then predict survival on the test set.  
Accuracy and classification report help assess model performance.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred))


## Step 9: Save the Trained Model

We save the trained logistic regression model to a file for future use without retraining.


In [ ]:
import joblib

joblib.dump(model, 'titanic_logistic_model.pkl')


In [ ]:
from google.colab import files
files.download('titanic_logistic_model.pkl')


### 🧮 Survival Count by Gender

This bar chart compares the number of passengers who survived and who did not, grouped by gender. It helps visualize how survival chances varied between male and female passengers.


In [ ]:
sns.countplot(x='Sex', hue='Survived', data=titanic_df)
plt.title('Survival Count by Gender')
plt.xlabel('Gender')
plt.ylabel('Passenger Count')
plt.legend(title='Survived', labels=['No', 'Yes'])
plt.show()

### 🚢 Survival Rate by Passenger Class

This bar plot shows the average survival rate for passengers in each class. It highlights the impact of socio-economic status on survival, with higher survival rates typically seen in 1st class.


In [ ]:
sns.barplot(x='Pclass', y='Survived', data=titanic_df)
plt.title('Survival Rate by Passenger Class')
plt.xlabel('Passenger Class')
plt.ylabel('Survival Rate')
plt.ylim(0, 1)
plt.show()


### 👶🧓 Survival Rate by Age Group

This bar chart displays the survival rates across different age categories. It helps us understand how age influenced the likelihood of survival.


In [ ]:
sns.barplot(x='AgeGroup', y='Survived', data=titanic_df, order=['Child', 'Teen', 'Adult', 'Senior', 'Elder'])
plt.title('Survival Rate by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Survival Rate')
plt.ylim(0, 1)
plt.show()


### 💰 Survival Rate by Fare Band

This chart shows how passengers paying higher fares generally had a better chance of survival. The FareBand ranges from 1 (lowest fare) to 4 (highest).


In [ ]:
sns.barplot(x='FareBand', y='Survived', data=titanic_df)
plt.title('Survival Rate by Fare Band')
plt.xlabel('Fare Band (1 = Lowest, 4 = Highest)')
plt.ylabel('Survival Rate')
plt.ylim(0, 1)
plt.show()

### 👶🧓 Survival Rate by Age Group

This bar plot displays the survival rate for different age groups. It provides insights into which age categories had higher survival chances during the Titanic disaster.


In [ ]:
sns.barplot(x='AgeGroup', y='Survived', data=titanic_df, order=['Child', 'Teen', 'Adult', 'Senior', 'Elder'])
plt.title('Survival Rate by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Survival Rate')
plt.ylim(0, 1)
plt.show()


# Titanic Data Analysis Project - Summary

In this project, I performed exploratory data analysis on the Titanic dataset to uncover patterns influencing passenger survival. Key factors such as age, fare, and family size showed significant impact on survival rates.

I built a logistic regression model which achieved about 80% accuracy in predicting survival, demonstrating the effectiveness of this approach.

This project highlights how data cleaning, feature engineering, visualization, and modeling come together in a typical data science workflow.

---

Thank You!
